In [3]:
import pandas as pd
import re

In [4]:
# pd.set_option('display.max_colwidth', None)

In [5]:
raw_df = pd.read_csv(r'C:\Users\niyai\git\british-analysis\data\British_Airway_Review.csv')

In [6]:
df = raw_df

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   reviews            2500 non-null   object
 1   date               2500 non-null   object
 2   country            2500 non-null   object
 3   seat_type          2500 non-null   object
 4   recommended        2500 non-null   object
 5   stars              2500 non-null   int64 
 6   route              2500 non-null   object
 7   type_of_traveller  2500 non-null   object
dtypes: int64(1), object(7)
memory usage: 156.4+ KB


In [8]:
df.head(1)

,reviews,date,country,seat_type,recommended,stars,route,type_of_traveller
0,✅ Trip Verified | I had the most fantastic BA...,1st August 2023,Hong Kong,Business Class,yes,5,Heathrow to Las Vegas,Family Leisure


In [9]:
df.recommended = df.recommended.str.capitalize()

In [10]:
df.recommended.unique()

array(['Yes', 'No'], dtype=object)

In [11]:
df.seat_type.unique()

array(['Business Class', 'Economy Class', 'Premium Economy',
       'First Class'], dtype=object)

In [12]:
df.type_of_traveller.unique()

array(['Family Leisure', 'Solo Leisure', 'Couple Leisure', 'Business'],
      dtype=object)

In [13]:
# Function to remove ordinal indicators from datetime strings
# Remove ordinal indicators from the 'date' column
def remove_ordinal_indicator(date_str):
    return re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)

df['date'] = df['date'].apply(remove_ordinal_indicator)

In [14]:
# Convert the 'date' column to datetime type
df['date'] = pd.to_datetime(df['date'], format='%d %B %Y')

In [15]:
df['ym'] = df['date'].astype(str)
df['ym'] = df['ym'].str[:7]

In [16]:
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [17]:
df['from'] = df['route'].str.split(' to ', expand = True)[0]
df['to'] = df['route'].str.split(' to ', expand = True)[1]

In [18]:
df['verified'] = df.reviews.str.startswith('✅ Trip Verified', '✅ Verified Review')

In [19]:
prefix = ( # verified
        '✅ Trip Verified'
          , '✅ Verified Review'
          # unverified
          , 'Not Verified'
          , '❎ Not Verified'
          , '❎ Unverified')

In [20]:
# custom function to check if a string starts with a prefix
def contains_prefix(row):
    return any(row['reviews'].startswith(prefix) for prefix in prefix)

# custom function to each row and store the result in a new column
df['contains_prefix'] = df.apply(contains_prefix, axis=1)


In [21]:
bool_to_yes_no = {True: 'Yes', False: 'No'}

# mapping dictionary to replace values in the type_traveller column
df['contains_prefix'] = df['contains_prefix'].map(bool_to_yes_no)
df['verified'] = df['verified'].map(bool_to_yes_no)

In [22]:
# Character after which you want to split the string
character ='|' 

# Custom function to split the string and select values after the character
def extract_value(value, character):
    if character in value:
        return value.split(character, 1)[-1]
    else:
        return value

In [23]:
df['cleaned_reviews'] = df['reviews'].apply(lambda x: extract_value(x, character))

In [24]:
newdf = df.drop(columns = ['reviews', 'route'])

In [28]:
newdf.to_csv('British_Airway_Review_cleaned.csv', index = False)

In [26]:
newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               2500 non-null   datetime64[ns]
 1   country            2500 non-null   object        
 2   seat_type          2500 non-null   object        
 3   recommended        2500 non-null   object        
 4   stars              2500 non-null   int64         
 5   type_of_traveller  2500 non-null   object        
 6   ym                 2500 non-null   object        
 7   day                2500 non-null   int32         
 8   month              2500 non-null   int32         
 9   year               2500 non-null   int32         
 10  from               2500 non-null   object        
 11  to                 2500 non-null   object        
 12  verified           2500 non-null   object        
 13  contains_prefix    2500 non-null   object        
 14  cleaned_